note: 这个代码是为了测试sentence embedding model的效果--将每个属性分开训练向量化模型

## 1 训练sentence embedding model，将元组转换为向量

### 1.1 加载数据，取出无关元组

In [1]:
import pandas as pd
import numpy as np
import string
import math
import matplotlib.pyplot as plt
import warnings
import random
warnings.filterwarnings("ignore")  #忽略告警
import os
import torch
from sentence_transformers import util
os.environ['CUDA_VISIBLE_DEVICES']='0'

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.5  # 程序最多只能占用指定gpu50%的显存
# config.gpu_options.allow_growth = True      #程序按需申请内存
# sess = tf.Session(config = config)

file_path = 'data/Structured/Amazon Google/'

table_a_df = pd.read_csv(file_path + 'tableA.csv')
table_b_df = pd.read_csv(file_path + 'tableB.csv')
train_df = pd.read_csv(file_path + 'train.csv')
valid_df = pd.read_csv(file_path + 'valid.csv')
test_df = pd.read_csv(file_path + 'test.csv')

In [2]:
train_df.shape, test_df.shape, valid_df.shape, table_a_df.shape, table_b_df.shape

((6874, 3), (2293, 3), (2293, 3), (1363, 4), (3226, 4))

## 2 构建AVB模型，无监督训练encoder,用于后续的分类模型

In [3]:
# 导入相关库
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import os
from tensorflow.contrib import slim
from tensorflow.contrib import layers as tflayers
from torch.autograd import Variable
import torchvision
import torch.utils.data as Data
import torch

from sklearn import metrics

In [4]:
# 定义全局超参数变量
global max_batch_n, batch_index
global LR_PRIMAL, LR_DUAL, X_dim, h_dim, z_dim, eps_dim, eps_nbasis
global encoder, decoder, discriminator

### 2.1 加载相关所需参数和函数

In [5]:
# 构造一个将所有句子，转换为tuple vector 的函数
def TupleEmbedding(all_sentences, sentence_embedding_model):
    '''
    all_sentences: 包含所有元组句子的列表
    sentence_embedding_model: 将句子转换为句子向量的模型
    founction：将所有的句子，转换为对应的sentence vector
    '''
    sentences_vector = []
    for sen in all_sentences:
        sentences_vector.append(sentence_embedding_model.encode(sen))
    return np.array(sentences_vector)

# 构造一个产生分批数据的函数
def GetBatchData(train_loader):
    '''
    传入torch的dataloader,然后生成对应的分批数据
    '''
    global batch_index
    for step, (batch_x, batch_y) in enumerate(train_loader):
        if step != batch_index:
            continue
        # batch_index = (batch_index + 1) % max_batch_n 
        # 对batch_x的维度进行修改，转换为（batch_size, -1）
        sample_x = batch_x.view(BATCH_SIZE, -1).data.numpy()
    return sample_x


def get_reconstr_err(de_x, x):
    x_multi = tf.multiply(x, x)
    de_x_multi = tf.multiply(de_x, de_x)
    x_de_x_multi = tf.multiply(x, de_x)

    x_mul_sum = tf.reduce_sum(x_multi)
    de_x_mul_sum = tf.reduce_sum(de_x_multi)
    x_de_mul_sum = tf.reduce_sum(x_de_x_multi)

    cosine = tf.divide(x_de_mul_sum, tf.add(x_mul_sum, de_x_mul_sum))
    reconst_err = tf.subtract(2., cosine)

    return reconst_err

### 2.2 构建编码器、解码器、判别器函数

In [6]:
## 构建编码器，解码器，判别器模型
def encoder_func(x):
    size = x.shape[0]
    # eps = tf.random_normal(tf.stack([eps_nbasis, size, eps_dim]))

    net = slim.fully_connected(x, h_dim, activation_fn=tf.nn.softplus)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    # net = slim.dropout(net, 0.3, is_training=False)
    # net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)

    z_mean = slim.fully_connected(net, z_dim, activation_fn=None)
    z_logstd = slim.fully_connected(net, z_dim, activation_fn=None)
    # logstd = log(std)
    return z_mean, z_logstd

def decoder_func(z):
    net = z
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.softplus)
    xlogits = slim.fully_connected(net, X_dim, activation_fn=None)
    return xlogits


### 2.3 构建模型训练计算图

In [7]:
## 构建计算图

def BuildComputationalGraph(x_real):
    '''
    构建计算图，输出两个损失值，和对应的参数
    '''
    z_sampled = tf.random_normal([BATCH_SIZE, z_dim])
    eps = tf.random_normal([BATCH_SIZE, z_dim])

    mean, logstd = encoder(x_real)
    std = tf.exp(logstd)
    z_real = mean + eps*std
    decoder_out = decoder(z_real)
    # # Primal loss
    reconst_err = get_reconstr_err(decoder_out, x_real)
    KL = tf.reduce_sum(0.5*tf.square(z_real) - logstd - 0.5, 1)
    loss_primal = tf.reduce_mean(reconst_err + KL)

    KL_mean = tf.reduce_mean(KL)
    reconst_err_mean = tf.reduce_mean(reconst_err)

    
    # 获取需要更新的参数
    qvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "encoder")
    pvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "decoder")


    return loss_primal, pvars+qvars,  reconst_err_mean, KL_mean

def GetTrainOp(loss_primal, loss_dual, vars_primal, vars_dual):
    learning_rate = LR_PRIMAL
    learning_rate_adversary = LR_DUAL

    # Train step
    primal_optimizer = tf.train.AdamOptimizer(learning_rate, use_locking=True, beta1=0.5)
    adversary_optimizer = tf.train.AdamOptimizer(learning_rate_adversary, use_locking=True, beta1=0.5)

    primal_grads = primal_optimizer.compute_gradients(loss_primal, var_list=vars_primal)
    adversary_grads = adversary_optimizer.compute_gradients(loss_dual, var_list=vars_dual)

    primal_grads = [(grad, var) for grad, var in primal_grads if grad is not None]
    adversary_grads = [(grad, var) for grad, var in adversary_grads if grad is not None]

    allgrads = [grad for grad, var in primal_grads + adversary_grads]
    with tf.control_dependencies(allgrads):
        primal_train_step = primal_optimizer.apply_gradients(primal_grads)
        adversary_train_step = adversary_optimizer.apply_gradients(adversary_grads)

    train_op = tf.group(primal_train_step, adversary_train_step)

    return train_op

### 2.4 模型训练

In [8]:
# global batch_index, h_dim
batch_index = 0

# 定义一些构建计算图所需函数
def Get2_WD(a_e, a_v, b_e, b_v):
    '''
    传入批量的两个多元高斯分布的均值和方差,(batch_size, z_dim)
    计算这一批多元高斯分布的平均距离,根据是否匹配，分别计算距离
    和对应的距离向量
    '''
    ab_e = tf.subtract(a_e, b_e)
    ab_v = tf.subtract(a_v, b_v)

    ab_e2 = tf.multiply(ab_e, ab_e)
    ab_v2 = tf.multiply(ab_v, ab_v)

    e_v = tf.add(ab_e2, ab_v2)
    e_v_sum = tf.reduce_mean(e_v, 1)

    return e_v_sum, e_v

# 构建二分类网络结构
def ClassifyModel(x):
    net = x
    h_dim = 300
    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)
    net = slim.dropout(net, 0.3, is_training=False)

    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)
    net = slim.dropout(net, 0.3, is_training=False)

    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)
    net = slim.dropout(net, 0.5, is_training=False)

    net = slim.fully_connected(net, h_dim, activation_fn=tf.nn.relu)

    prediction = slim.fully_connected(net, 2, activation_fn=tf.nn.softmax)
    return prediction

classify_model = tf.make_template('ClassifyModel', ClassifyModel)

In [9]:
# # 保存X矩阵的内容
# X = sentences_vector
# with open('X_array1_10.txt', 'w') as fp:
#   for item in X:
#     temp = [str(i) for i in item]
#     s = ' '.join(temp)
#     fp.write(s + '\n')

In [10]:
# 读取保存的X矩阵
X = []
with open(file_path + 'X_array0_100.txt', 'r') as fp:
    temp = fp.readlines()
for item in temp:
    s = item.strip('\n')
    slist = s.split()
    slist = [eval(i) for i in slist]
    X.append(slist)
X = np.array(X)
X = torch.FloatTensor(X).data.numpy()
print(X.shape)
sentences_vector = X

(4589, 768)


In [11]:
# # 提取训练集合测试集当中的数据进行训练SE模型
# a_id_list = []
# b_id_list = []
# for i in range(train_df.shape[0]):
#     a_id_list.append(train_df.loc[i, 'ltable_id'])
#     b_id_list.append(train_df.loc[i, 'rtable_id'])
# for i in range(test_df.shape[0]):
#     a_id_list.append(test_df.loc[i, 'ltable_id'])
#     b_id_list.append(test_df.loc[i, 'rtable_id'])
# a_id_list = list(set(a_id_list))
# b_id_list = list(set(b_id_list))
    
# X_train_test = []
# for a_id in a_id_list:
#     X_train_test.append(sentences_vector[a_id])
# for b_id in b_id_list:
#     X_train_test.append(sentences_vector[table_a_df.shape[0] + b_id])

# X_train_test = np.array(X_train_test)
# X_train_test = torch.FloatTensor(X_train_test).data.numpy()
# print(X_train_test.shape)

In [12]:
## 构建批处理操作
BATCH_SIZE = 20
Y_fake = torch.linspace(1, 100, X.shape[0]) # 为X_attr 构建一个虚假的Y值
torch_dataset = Data.TensorDataset(torch.FloatTensor(X), Y_fake)

train_loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

## 定义超参数
X_dim = X.shape[1]
h_dim = 300
z_dim = 128
eps_dim = 32
eps_nbasis = 16

# 获取模型变量
encoder = tf.make_template('encoder', encoder_func)
decoder = tf.make_template('decoder', decoder_func)

# 构建计算图
LR = 1e-5

x_real = tf.placeholder(tf.float32, shape=[BATCH_SIZE, X_dim])
loss_primal, vars_primal,  reconst_err_mean, KL_mean = BuildComputationalGraph(x_real)
optimizer = tf.train.AdamOptimizer(LR)
train_op1 = optimizer.minimize(loss_primal, var_list=vars_primal)

Instructions for updating:
Please use `layer.__call__` method instead.


In [13]:
sentences_vector = X
# 先给train_df添加一个属性列similar:表示两个元组对应向量之间的余弦相似度
train_df['similar'] = 0.0

# 计算出train_df中所有元组对，对应的余弦相似度
for i in range(train_df.shape[0]):
    a_id = train_df.loc[i, 'ltable_id']
    b_id = train_df.loc[i, 'rtable_id']
    em_a = sentences_vector[a_id]
    em_b = sentences_vector[table_a_df.shape[0] + b_id]
    train_df.loc[i, 'similar'] = float(util.cos_sim(em_a, em_b))
    
pairs_0 = train_df[train_df['label'] == 0].values.tolist()
pairs_1 = train_df[train_df['label'] == 1].values.tolist()

# 设置批处理函数所需的，索引
global id_0, id_1, num_0, num_1
id_0, id_1 = 0, 0
num_0, num_1 = 15, 5
total_num01 = num_0 + num_1

# 构建新的生产批数据的函数
def GetBatchData_2(pairs_0, pairs_1, sentences_vector):
    '''
    新的批数据生产函数
    '''
    # 根据batch_size设置pairs_0和pairs_1，每次抽样多少个数据
    # pairs_0 每批次15个，pairs_1 每批次5个
    global id_0, id_1, num_0, num_1
    
    len_0, len_1 = len(pairs_0), len(pairs_1)
#     move_0 = len_0 // num_0
#     move_1 = len_1 // num_1
#     move_0, move_1 = 100, 50
    x_a, x_b, y = [], [], []
    
    # 对pairs_0进行采样
    for i in range(num_0):
        id_0 = (id_0 + 1) % len_0
        x_a.append(sentences_vector[ int(pairs_0[id_0][0]) ])
        x_b.append(sentences_vector[table_a_df.shape[0] + int(pairs_0[id_0][1]) ])
        y.append(int(pairs_0[id_0][2]))
        
    for i in range(num_1):
        id_1 = (id_1 + 1) % len_1
        x_a.append(sentences_vector[ int(pairs_1[id_1][0]) ])
        x_b.append(sentences_vector[table_a_df.shape[0] + int(pairs_1[id_1][1]) ])
        y.append(int(pairs_1[id_1][2]) )
    
    x_a = np.array(x_a).reshape(total_num01, -1)
    x_b = np.array(x_b).reshape(total_num01, -1)
    y = np.array(y).reshape(total_num01, -1)
    
    return x_a, x_b, y



In [14]:
def TrainClassifyModel(x_a, x_b, y):
    # 对数据进行编码处理
    ez_a, logstd_a = encoder(x_a)
    ez_b, logstd_b = encoder(x_b)
    # 这里的vz 表示z的标准差（不修改名字了，因该是std_z_a）
    vz_a = tf.exp(logstd_a)
    vz_b = tf.exp(logstd_b)

    # 计算2-WassersteinDistance, (batch_size, 1), (batch_size, z_dim)
    WD_disc, WD_vector = Get2_WD(ez_a, vz_a, ez_b, vz_b)

    # 计算 模型的预测,每组数据对应匹配的概率，（batch_size, 2）
    # x_data = tf.concat([x_a, x_b], axis=-1)
    prediction = classify_model(WD_vector)
    predict_0 = tf.slice(prediction, [0,0],[-1,1])
    predict_1 = tf.slice(prediction, [0,1],[-1,1])

    # 构造对比损失函数-论文当中的,Cost-effictive ...
    M = 0.5  # 调节encoder参数更新的超参数
    class_w = 1.0  # 增加匹配样例的权重
    # 对概率值进行范围限制，防止log()操作出现nan
    predict_1 = tf.clip_by_value(predict_1, 1e-8, 1.0)
    predict_0 = tf.clip_by_value(predict_0, 1e-8, 1.0)
    loss_1 = -tf.add(tf.multiply(3*y, tf.log(predict_1)), tf.multiply(1*(1.0-y), tf.log(predict_0)))
    loss_2 = tf.add(tf.multiply(3*y, WD_disc), tf.multiply(1.0-y, tf.maximum(0.0,M-WD_disc)))

    loss_1 = tf.reduce_mean(loss_1)
    loss_2 = tf.reduce_mean(loss_2)
    total_loss = loss_1 + loss_2

    # 获取 模型需要更新的参数
    encoder_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "encoder")
    classify_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "ClassifyModel")

    # optimizer = tf.train.AdamOptimizer(LR)
    # loss1_op = optimizer.minimize(loss_1, var_list=classify_vars+encoder_vars)
    # loss2_op = optimizer.minimize(loss_2, var_list=encoder_vars)
    train_op2 = GetTrainOp(2*loss_1, 1*loss_2, encoder_vars+classify_vars, encoder_vars)
    # train_op = optimizer.minimize(total_loss, var_list=encoder_vars+classify_vars)
    return train_op2, loss_1, loss_2

LR_PRIMAL = 2e-5
LR_DUAL = 1e-5

x_a = tf.placeholder(tf.float32, shape=[total_num01, X_dim])
x_b = tf.placeholder(tf.float32, shape=[total_num01, X_dim])
y = tf.placeholder(tf.float32, shape=[total_num01, 1])
train_op2, loss_1, loss_2 = TrainClassifyModel(x_a, x_b, y)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
#训练计算图
sess = tf.InteractiveSession()  # 创建会话交互类
sess.run(tf.global_variables_initializer())  # 初始化变量

batch_index = 0
max_batch_n = X.shape[0] // BATCH_SIZE
progress = tqdm_notebook(range(60*max_batch_n))
for i in progress:
    x = GetBatchData(train_loader)
    batch_index = (batch_index + 1) % max_batch_n

    # sess.run(train_op1)
    _,loss_p, r_e,kl = sess.run([train_op1,loss_primal,reconst_err_mean,KL_mean], feed_dict={x_real:x})
    progress.set_description('loss_primal= %.4f, re=%.4f - ' % (loss_p,r_e))

# # 保存对应的参数
# saver = tf.train.Saver()
# saver.save(sess,'AVB_paramter.ckpt')

  0%|          | 0/13740 [00:00<?, ?it/s]

*   loss_primal= 0.0417, loss_dual= 3.6338, re=1.8010 
*   loss_primal= 0.0386, loss_dual= 3.9267, re=1.8149

In [18]:
# 获取模型测试用的test数据
sentences_vector = X
X_a = []
X_b = []
## valid_df，转换为test_df
for i in range(test_df.shape[0]):
    a_id = test_df.loc[i, 'ltable_id']
    b_id = test_df.loc[i, 'rtable_id']
    X_a.append(sentences_vector[a_id])
    X_b.append(sentences_vector[table_a_df.shape[0] + b_id])
X_a = np.array(X_a)
X_b = np.array(X_b)

Y = test_df['label'].values

In [19]:
# 训练过程
# x_data = np.concatenate([X_a, X_b], axis=-1)
id_0, id_1 = 0, 0
max_batch_n_2 = train_df.shape[0] // total_num01
epoch_n = 2000 * max_batch_n_2
progress = tqdm_notebook(range(epoch_n))
LR_PRIMAL = 2e-4
LR_DUAL = 1e-5
for i in progress:
    # 更新参数
    # sess.run(train_op2)
    Xa, Xb, Yy = GetBatchData_2(pairs_0, pairs_1, sentences_vector)
    id_0 += 1
    id_1 += 1

    _, loss1, loss2 = sess.run([train_op2, loss_1, loss_2], feed_dict={x_a:Xa, x_b:Xb, y:Yy})
    progress.set_description('loss1 = %.5f, loss2 = %.4f - '%(loss1,loss2))

    num_0 = random.randint(8,total_num01)
    num_1 = total_num01 - num_0      

    # 用测试集进行验证
    if (i < max_batch_n_2 * 500):
        temp = 7*max_batch_n_2
    else:
        temp = max_batch_n_2
#     temp = 10
    if i % temp == 0:
        e_a, v_a = encoder(X_a)
        e_b, v_b = encoder(X_b)
        
        v_a = tf.exp(v_a)
        v_b = tf.exp(v_b)
        
        wd_disc, wd_vector = Get2_WD(e_a, v_a, e_b, v_b)

        # sess.run(wd_vector)
        output = classify_model(wd_vector)
        # output = tf.argmax(output, 1)
        output = sess.run(output)
        out = np.argmax(output, 1)
        # 计算交叉熵误差
        # accuracy = sum(out == Y) / X_a.shape[0]
        # print('epoch = %d, accuracy = %.4f' % (i, accuracy))
        # 计算模型效果
        precision = metrics.precision_score(Y, out)
        recall = metrics.recall_score(Y, out)
        f1_score = metrics.f1_score(Y, out)
#         if f1_score > 0.60:
        print('epoch = %d, precision = %.3f, recall = %.3f, f1_score = %.4f'%(i,precision,recall,f1_score))


  0%|          | 0/686000 [00:00<?, ?it/s]

epoch = 0, precision = 0.043, recall = 0.222, f1_score = 0.0718
epoch = 2401, precision = 0.522, recall = 0.620, f1_score = 0.5664
epoch = 4802, precision = 0.576, recall = 0.603, f1_score = 0.5887
epoch = 7203, precision = 0.618, recall = 0.504, f1_score = 0.5553
epoch = 9604, precision = 0.567, recall = 0.611, f1_score = 0.5885
epoch = 12005, precision = 0.609, recall = 0.585, f1_score = 0.5969
epoch = 14406, precision = 0.687, recall = 0.479, f1_score = 0.5642
epoch = 16807, precision = 0.587, recall = 0.547, f1_score = 0.5664
epoch = 19208, precision = 0.604, recall = 0.581, f1_score = 0.5926
epoch = 21609, precision = 0.636, recall = 0.509, f1_score = 0.5653
epoch = 24010, precision = 0.624, recall = 0.526, f1_score = 0.5708
epoch = 26411, precision = 0.624, recall = 0.568, f1_score = 0.5951
epoch = 28812, precision = 0.624, recall = 0.581, f1_score = 0.6018
epoch = 31213, precision = 0.670, recall = 0.504, f1_score = 0.5756
epoch = 33614, precision = 0.668, recall = 0.585, f1_sco

epoch = 187964, precision = 0.762, recall = 0.479, f1_score = 0.5879
epoch = 188307, precision = 0.631, recall = 0.607, f1_score = 0.6187
epoch = 188650, precision = 0.758, recall = 0.496, f1_score = 0.5995
epoch = 188993, precision = 0.769, recall = 0.470, f1_score = 0.5836
epoch = 189336, precision = 0.769, recall = 0.470, f1_score = 0.5836
epoch = 189679, precision = 0.767, recall = 0.479, f1_score = 0.5895
epoch = 190022, precision = 0.759, recall = 0.457, f1_score = 0.5707
epoch = 190365, precision = 0.737, recall = 0.491, f1_score = 0.5897
epoch = 190708, precision = 0.747, recall = 0.466, f1_score = 0.5737
epoch = 191051, precision = 0.780, recall = 0.423, f1_score = 0.5485
epoch = 191394, precision = 0.774, recall = 0.440, f1_score = 0.5613
epoch = 191737, precision = 0.769, recall = 0.385, f1_score = 0.5128
epoch = 192080, precision = 0.758, recall = 0.389, f1_score = 0.5141
epoch = 192423, precision = 0.758, recall = 0.389, f1_score = 0.5141
epoch = 192766, precision = 0.846,

epoch = 228781, precision = 0.703, recall = 0.585, f1_score = 0.6387
epoch = 229124, precision = 0.761, recall = 0.504, f1_score = 0.6067
epoch = 229467, precision = 0.747, recall = 0.543, f1_score = 0.6287
epoch = 229810, precision = 0.732, recall = 0.573, f1_score = 0.6427
epoch = 230153, precision = 0.778, recall = 0.449, f1_score = 0.5691
epoch = 230496, precision = 0.797, recall = 0.436, f1_score = 0.5635
epoch = 230839, precision = 0.719, recall = 0.590, f1_score = 0.6479
epoch = 231182, precision = 0.786, recall = 0.393, f1_score = 0.5242
epoch = 231525, precision = 0.761, recall = 0.462, f1_score = 0.5745
epoch = 231868, precision = 0.770, recall = 0.457, f1_score = 0.5737
epoch = 232211, precision = 0.776, recall = 0.487, f1_score = 0.5984
epoch = 232554, precision = 0.798, recall = 0.423, f1_score = 0.5531
epoch = 232897, precision = 0.707, recall = 0.496, f1_score = 0.5829
epoch = 233240, precision = 0.740, recall = 0.474, f1_score = 0.5781
epoch = 233583, precision = 0.724,

epoch = 269598, precision = 0.772, recall = 0.419, f1_score = 0.5429
epoch = 269941, precision = 0.800, recall = 0.410, f1_score = 0.5424
epoch = 270284, precision = 0.808, recall = 0.415, f1_score = 0.5480
epoch = 270627, precision = 0.782, recall = 0.415, f1_score = 0.5419
epoch = 270970, precision = 0.761, recall = 0.462, f1_score = 0.5745
epoch = 271313, precision = 0.782, recall = 0.415, f1_score = 0.5419
epoch = 271656, precision = 0.797, recall = 0.402, f1_score = 0.5341
epoch = 271999, precision = 0.800, recall = 0.410, f1_score = 0.5424
epoch = 272342, precision = 0.785, recall = 0.406, f1_score = 0.5352
epoch = 272685, precision = 0.787, recall = 0.410, f1_score = 0.5393
epoch = 273028, precision = 0.808, recall = 0.415, f1_score = 0.5480
epoch = 273371, precision = 0.739, recall = 0.509, f1_score = 0.6025
epoch = 273714, precision = 0.733, recall = 0.517, f1_score = 0.6065
epoch = 274057, precision = 0.742, recall = 0.491, f1_score = 0.5913
epoch = 274400, precision = 0.734,

epoch = 310415, precision = 0.774, recall = 0.440, f1_score = 0.5613
epoch = 310758, precision = 0.721, recall = 0.530, f1_score = 0.6108
epoch = 311101, precision = 0.740, recall = 0.487, f1_score = 0.5876
epoch = 311444, precision = 0.769, recall = 0.483, f1_score = 0.5932
epoch = 311787, precision = 0.779, recall = 0.466, f1_score = 0.5829
epoch = 312130, precision = 0.740, recall = 0.534, f1_score = 0.6203
epoch = 312473, precision = 0.773, recall = 0.466, f1_score = 0.5813
epoch = 312816, precision = 0.759, recall = 0.432, f1_score = 0.5504
epoch = 313159, precision = 0.713, recall = 0.543, f1_score = 0.6165
epoch = 313502, precision = 0.717, recall = 0.530, f1_score = 0.6093
epoch = 313845, precision = 0.729, recall = 0.530, f1_score = 0.6139
epoch = 314188, precision = 0.726, recall = 0.487, f1_score = 0.5831
epoch = 314531, precision = 0.759, recall = 0.457, f1_score = 0.5707
epoch = 314874, precision = 0.822, recall = 0.355, f1_score = 0.4955
epoch = 315217, precision = 0.745,

epoch = 351232, precision = 0.729, recall = 0.517, f1_score = 0.6050
epoch = 351575, precision = 0.771, recall = 0.462, f1_score = 0.5775
epoch = 351918, precision = 0.770, recall = 0.457, f1_score = 0.5737
epoch = 352261, precision = 0.741, recall = 0.526, f1_score = 0.6150
epoch = 352604, precision = 0.753, recall = 0.483, f1_score = 0.5885
epoch = 352947, precision = 0.733, recall = 0.517, f1_score = 0.6065
epoch = 353290, precision = 0.736, recall = 0.453, f1_score = 0.5608
epoch = 353633, precision = 0.719, recall = 0.513, f1_score = 0.5985
epoch = 353976, precision = 0.707, recall = 0.526, f1_score = 0.6029
epoch = 354319, precision = 0.714, recall = 0.564, f1_score = 0.6301
epoch = 354662, precision = 0.742, recall = 0.504, f1_score = 0.6005
epoch = 355005, precision = 0.726, recall = 0.453, f1_score = 0.5579
epoch = 355348, precision = 0.759, recall = 0.457, f1_score = 0.5707
epoch = 355691, precision = 0.689, recall = 0.568, f1_score = 0.6230
epoch = 356034, precision = 0.768,

epoch = 392049, precision = 0.744, recall = 0.521, f1_score = 0.6131
epoch = 392392, precision = 0.739, recall = 0.509, f1_score = 0.6025
epoch = 392735, precision = 0.676, recall = 0.598, f1_score = 0.6349
epoch = 393078, precision = 0.739, recall = 0.509, f1_score = 0.6025
epoch = 393421, precision = 0.699, recall = 0.577, f1_score = 0.6323
epoch = 393764, precision = 0.778, recall = 0.419, f1_score = 0.5444
epoch = 394107, precision = 0.726, recall = 0.543, f1_score = 0.6210
epoch = 394450, precision = 0.750, recall = 0.526, f1_score = 0.6181
epoch = 394793, precision = 0.759, recall = 0.526, f1_score = 0.6212
epoch = 395136, precision = 0.760, recall = 0.556, f1_score = 0.6420
epoch = 395479, precision = 0.769, recall = 0.483, f1_score = 0.5932
epoch = 395822, precision = 0.810, recall = 0.419, f1_score = 0.5521
epoch = 396165, precision = 0.824, recall = 0.359, f1_score = 0.5000
epoch = 396508, precision = 0.754, recall = 0.538, f1_score = 0.6284
epoch = 396851, precision = 0.652,

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[2293,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node encoder_1593/fully_connected_1/MatMul (defined at /export/home/lsc/anaconda3/envs/tensorflow1.x/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[ClassifyModel_796/fully_connected_4/Softmax/_1609]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[2293,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node encoder_1593/fully_connected_1/MatMul (defined at /export/home/lsc/anaconda3/e